In [17]:
stt_pecha_tools = "/home/monlamai/Documents/GitHub/combining-dataset/stt_pecha.tsv"
prodigy = "/home/monlamai/Documents/GitHub/combining-dataset/prodigy.tsv"
mv = "/home/monlamai/Documents/GitHub/saymore-report-generator/elan_to_segments/stt_mv.tsv"

In [18]:
old_mv_test = "/home/monlamai/Downloads/test.tsv"
old_mv_train = "/home/monlamai/Downloads/train.tsv"
old_mv_validation = "/home/monlamai/Downloads/validation.tsv"

In [19]:
import pandas as pd
old_mv_test_df = pd.read_csv(old_mv_test, sep='\t')
old_mv_train_df = pd.read_csv(old_mv_train, sep='\t')
old_mv_validation_df = pd.read_csv(old_mv_validation, sep='\t')

In [20]:
import pandas as pd
pecha_tools_df = pd.read_csv(stt_pecha_tools, sep='\t')
prodigy_df = pd.read_csv(prodigy, sep='\t')
mv_df = pd.read_csv(mv, sep='\t')

In [21]:
df = pd.concat([pecha_tools_df, prodigy_df, mv_df, old_mv_test_df, old_mv_train_df, old_mv_validation_df], ignore_index=True)

# clean the combined tsv

In [22]:
import re
text = "ཧ་ཧ་ཧ་ཧ་ཧ་འེ་"
text = re.sub(r'ཧ་ཧ་(ཧ་)+', r'ཧ་ཧ་ཧ་', text)
print(text)

ཧ་ཧ་ཧ་འེ་


In [23]:
import re
def clean_transcription(text):
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = text.strip()

    text = re.sub(r"་+", "་", text)
    text = re.sub(r"།+", "།", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\s+།", "།", text)

    text = re.sub(r"ཧཧཧ+", "ཧཧཧ", text)
    text = re.sub(r'འེ་འེ་(འེ་)+', r'འེ་འེ་འེ་', text)
    text = re.sub(r'ཧ་ཧ་(ཧ་)+', r'ཧ་ཧ་ཧ་', text)

    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\/\{\}\(\)\༽\》\༼\《\༅\༄\༈\༑\༠]'
    
    text = re.sub(chars_to_ignore_regex, '', text)+" "
    return text

def check_if_regex(text):
    text = str(text)
    regex = re.compile(r'[a-zA-Z]+')
    match = re.search(regex, text)
    
    # Check if a match was found.
    if match:
        return True
    else:
        return False

In [24]:
df.shape

(661867, 5)

In [25]:
df = df[df['uni'].str.len() > 0]

In [26]:
df = df[df['uni'].str.len() < 994]

In [27]:
df = df[~df['uni'].apply(check_if_regex)]

In [28]:
df.shape

(661393, 5)

In [29]:
df['uni'] = df['uni'].map(clean_transcription)

In [30]:
import pyewts
converter = pyewts.pyewts()

df['wylie'] = df['uni'].map(converter.toWylie)

In [31]:
df.head()

,file_name,uni,wylie,url,dept
0,STT_AB00156_0296_1211329_to_1212494,བོད་གཞུང་བཀའ་ཤག་དང༌,bod gzhung bka' shag dang*_,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
1,STT_AB00157_0050_200635_to_201377,དེར་བརྟེན་ན་,der brten na _,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
2,STT_AB00156_0239_937026_to_938157,ཅེས་ཟེར་ཡང༌།,ces zer yang*/_,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
3,STT_AB00156_0240_938579_to_944485,བོད་ཤར་ཕྱོགས་ས་ཁུལ་དུ་མི་མང་གི་རེ་འདོད་གསལ་པོ་...,bod shar phyogs sa khul du mi mang gi re 'dod ...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
4,STT_AB00156_0241_944974_to_946307,བྱས་དང་བྱེད་བཞིན་ཡོད་པ་རེད།,byas dang byed bzhin yod pa red/_,https://huggingface.co/datasets/openpecha/STT_...,STT_AB


In [32]:
df['url'] = df['url'].map(lambda x : x.replace('#','%23'))

In [33]:
df = df[df['file_name'] != 'STT_AB00148_0687_2124469_to_2126579']

In [34]:
df.drop_duplicates(subset='file_name', keep="first", inplace=True)

In [35]:
df.to_csv('all.tsv', sep='\t', index=False)

In [36]:
df.head()

,file_name,uni,wylie,url,dept
0,STT_AB00156_0296_1211329_to_1212494,བོད་གཞུང་བཀའ་ཤག་དང༌,bod gzhung bka' shag dang*_,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
1,STT_AB00157_0050_200635_to_201377,དེར་བརྟེན་ན་,der brten na _,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
2,STT_AB00156_0239_937026_to_938157,ཅེས་ཟེར་ཡང༌།,ces zer yang*/_,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
3,STT_AB00156_0240_938579_to_944485,བོད་ཤར་ཕྱོགས་ས་ཁུལ་དུ་མི་མང་གི་རེ་འདོད་གསལ་པོ་...,bod shar phyogs sa khul du mi mang gi re 'dod ...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
4,STT_AB00156_0241_944974_to_946307,བྱས་དང་བྱེད་བཞིན་ཡོད་པ་རེད།,byas dang byed bzhin yod pa red/_,https://huggingface.co/datasets/openpecha/STT_...,STT_AB


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Splitting the data
train, test_temp = train_test_split(df, test_size=0.1, random_state=42, stratify=df['dept'])
val, test = train_test_split(test_temp, test_size=0.5, random_state=42, stratify=test_temp['dept'])         

In [38]:
len(train), len(val), len(test), len(train)+len(val)+len(test), len(df)

(578333, 32130, 32130, 642593, 642593)

In [39]:
train.to_csv('train.tsv', sep='\t', index=False)
val.to_csv(  'val.tsv', sep='\t', index=False)
test.to_csv( 'test.tsv', sep='\t', index=False)

In [40]:
df['uni'].str.contains('ཧཧཧ').value_counts()

uni
False    642034
True        559
Name: count, dtype: int64

In [1]:
import pandas as pd

df = pd.read_csv('all.tsv', sep='\t')

In [50]:
import os
def getTimeSpan(filename):
    filename = os.path.splitext(os.path.basename(filename))[0]
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return abs((end - start)/1000)
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return abs(end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0


In [51]:
df['time_span'] = df['file_name'].map(getTimeSpan)

In [52]:
df.groupby('dept')['time_span'].sum()/60/60

dept
STT_AB     83.979735
STT_CS     68.579719
STT_MV    307.320387
STT_NS     97.503928
STT_TT    141.066895
Name: time_span, dtype: float64

In [53]:
df['time_span'].sum()/60/60

698.4506650000003

In [57]:
# df.sort_values(by='time_span', ascending=False, inplace=True)
df = df.sample(frac = 1)

In [76]:
df.iloc[0:100,[0, 1, 3, 5]].to_csv("random_100.csv", index=False)
df.iloc[0:100,[0, 1, 3, 5]]

,file_name,uni,url,time_span
209213,STT_TT00012_00268.450-00270.850,དབུ་མའི་གཞུང་གསུང་པའི་ཁ་ཐུག་ལ་ཡ། ཨེ་ནས།,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,1.550
403296,STT_TT00122_01016.350-01022.200,སྙིང་རྗེ་ཟེར་ཡ་འདི་ང་རང་ཚོའི་ད་ལྟ་སོ་སོའི་གཅིག...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,5.650
224167,STT_TT00016_07023.050-07026.100,འཕགས་པ་ཐམས་ཅད་ཡོངས་སུ་འཕགས་པ་ཐམས་ཅད་ཡོད་པར་སྨྲ...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,2.950
604284,STT_MV0004_0148_1342729_to_1347239.wav,སི་ཧའི།,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,4.510
473125,STT_TT00107_05395.200-05396.450,བློ་མི་ཤེས་སེམས་ཅན་,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,0.800
...,...,...,...,...
31895,STT_TT00505_0197_1670196_to_1672329.wav,སྒོ་བཞི་ནི་གཡང་བྱས།,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,2.133
396110,STT_TT00022_06998.500-07002.900,ཆོས་རྣམས་རང་བཞིན་མེད་པར་ཡོངས་སུ་ཤེས་གྱུར་པ།།འད...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,3.500
598012,STT_MV0264_0145_973472_to_976121.wav,བརྩེ་དུང་ཐོག་རེད།་གཞན་དག་དེ།,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,2.649
406813,STT_TT00129_01262.750-01265.600,དྭ་ད་ག་ནང་བཞིན་ནད་ན་ཚ་ཏིག་ཙི་བྱུང་བསྡད་ཡོད་ནས་...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,2.250
